In [ ]:
##################################################
### Author: Anthony Igel                       ###
### Team: Category Management Transformation   ###
### Project: Developing practical Python Tools ###
### Purpose: Logistic Regression               ###
### Date: 05/24/2018                           ###
##################################################

######################################################################
########                     Import Modules                   ########
######################################################################
import py_effo as py_effo

### pandas
# Pandas is for structured data operations and manipulations, extensively used for data preparation
import pandas as pd

### numpy
# NumPy stands for Numerical Python, a library contains basic linear algebra functions, Fourier Transforms and advanced random
# number capabilities
import numpy as np 

### Scipy
# Scipy performs a host of statistical calculations, built on top of Numpy, thus we do not need to import Numpy as all Numpy
# functions are contained in Scipy
# https://oneau.wordpress.com/2011/02/28/simple-statistics-with-scipy/
import scipy as sp

### sklearn
# Sklearn contains basic statistical models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection  import train_test_split
from sklearn.model_selection  import cross_val_score
# As well as a module to calculate model performance statistics
from sklearn import metrics

### Statsmodels
# Sklearn contains basic statistical models and data sets
import statsmodels.api as sm

### Matplotlib
# Matplotlib is a Python based plotting library with complete 2D support and limited 3D support
%matplotlib inline
import matplotlib as mlb
import matplotlib.pyplot as plt

### Seaborn
# Seaborn is a Python visualization library based on Matplolib, providing high-level interface for statistcial graphing
# Seaborn supports numpy and pandas data structures as well as statistical routines from scipy and statsmodels
# Note: https://seaborn.pydata.org/introduction.html
import seaborn as sns

### String
# Allows for more flexible solutions for dealing with string characters
import string as st

### Patsy.dmatrices
# Python package for describing statistical models (especially linear models, or models that have a linear component) 
# and building design matrices
from patsy import dmatrices

In [ ]:
######################################################################
########                     Import Data                      ########
######################################################################

### Advertising Data
# load dataset from statsmodel modules
dta = sm.datasets.fair.load_pandas().data

### We need to add a dummy variable to indicate whether of not an affair occurred
### 1 indicates having an affair, 0 represents not
dta['affair'] = (dta.affairs > 0).astype(int)

######################################################################
########                  Data Expoloration                   ########
######################################################################
### Identify any null values in data set
print('Data Type Information of Affair Data')
print(dta.info())
print()
print()
### Identify any null values in data set
print('Summary of Nulls in Affair Data')
print(dta.isnull().sum())
print()
print()
### Let's determine how the data types look for this data frame
print('View Mean of Variables, Grouped by Affair Flag')
print(dta.groupby('affair').mean())
print()
print()
print('View Mean of Variables, Grouped by Rate_Marriage')
print(dta.groupby('rate_marriage').mean())

### From this, we can make a few assertions
# 1. On average, women who have affairs rate their marriages lower
# 2. An increase in age, yrs_married and children appear to correlate with a declining marriage rating

In [ ]:
######################################################################
########                  Data Visualization                  ########
######################################################################
# show plots in the notebook
%matplotlib inline

# histogram of education vs marriage rating
dta.educ.hist()
plt.title('Histogram of Education')
plt.xlabel('Education Level')
plt.ylabel('Frequency')
plt.show()

print()

# histogram of marriage rating
dta.rate_marriage.hist()
plt.title('Histogram of Marriage Rating')
plt.xlabel('Marriage Rating')
plt.ylabel('Frequency')
plt.show()

print()

# barplot of marriage rating grouped by affair (True or False)
pd.crosstab(dta.rate_marriage, dta.affair.astype(bool)).plot(kind='bar')
plt.title('Marriage Rating Distribution by Affair Status')
plt.xlabel('Marriage Rating')
plt.ylabel('Frequency')
plt.show()

print()

# stacked barplot  of women having affairs by number of years of marriage
affair_yrs_married = pd.crosstab(dta.yrs_married, dta.affair.astype(bool))
affair_yrs_married.div(affair_yrs_married.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Affair Percentage by Years Married')
plt.xlabel('Years Married')
plt.ylabel('Percentage')
plt.show()

In [ ]:
######################################################################
########                  Data Preparation                    ########
######################################################################

### This will ensure that all column names are stripped of whitespace
dta.rename(columns = lambda x: x.strip(), inplace = True)

### We can also adjust the case of our metrics table columns
dta.rename(columns = lambda x: x.lower(), inplace = True)

### We will want to add an intercept column as well as dummy variables for occupation and occupation_husb, since they are 
### categorical variables

### The dmatricies function from the patsy module will help create dataframes with an intercept column and dummy variables for
# occupation and occupation_husb
y, x = dmatrices('affair ~ rate_marriage + age + yrs_married + children + \
                  religious + educ + C(occupation) + C(occupation_husb)',
                  dta, return_type = "dataframe")
print(x.columns)

### Fix column names for the dummy variables 

x = x.rename(columns = {'C(occupation)[T.2.0]':'occ_2',
                        'C(occupation)[T.3.0]':'occ_3',
                        'C(occupation)[T.4.0]':'occ_4',
                        'C(occupation)[T.5.0]':'occ_5',
                        'C(occupation)[T.6.0]':'occ_6',
                        'C(occupation_husb)[T.2.0]':'occ_husb_2',
                        'C(occupation_husb)[T.3.0]':'occ_husb_3',
                        'C(occupation_husb)[T.4.0]':'occ_husb_4',
                        'C(occupation_husb)[T.5.0]':'occ_husb_5',
                        'C(occupation_husb)[T.6.0]':'occ_husb_6'})
print()
print(x.columns)
### Additionally, we need to flatten our dependent variable into a 1-D arrary so scikit-learn will properly understand it
y = np.ravel(y)

In [ ]:
######################################################################
########                     Modeling                         ########
######################################################################

######## Logistic Regression ########

### Lets run a logistic regression on the entire data set and see if it accurate
# instantiate a logistic regression model, and fit with X and y
model = LogisticRegression()
model = model.fit(x, y)

# check the accuracy on the training set
ModelScore_1 = model.score(x, y)
print('Model Score: ' + str((ModelScore_1 * 100).round(0)) + '%')
print()
# what percentage had affairs?
PredMean_1 = y.mean()
print('What % of respondents had an affair? '+ str((PredMean_1 * 100).round(0)) + '%')

In [ ]:
######## Data Splitting ########

### We want to split our independent and dependent data sets into training and testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

print('Dependent Variables')
print('Training: ' + str(len(x_train)))
print('Testing: ' + str(len(x_test)))
print()
print('Independent Variables')
print('Training: ' + str(len(y_train)))
print('Testing: ' + str(len(y_test)))

In [ ]:
######################################################################
########                     Modeling                         ########
######################################################################

######## Logistic Regression ########
model2 = LogisticRegression()
model2.fit(x_train, y_train)

### Predict class labels for the test set
predicted = model2.predict(x_test)

print('Predicted Values')
print(predicted)
print()
### Generate class probabilities
probs = model2.predict_proba(x_test)
print('Prediction Probabilities')
print(probs)
print()

### Generate evaluation metrics by using our predicted values compared to the truth
AccuracyScore_2 = metrics.accuracy_score(y_test, predicted)
RocAucScore_2 = metrics.roc_auc_score(y_test, probs[:, 1])
print('Accuracy Score: ' + str((AccuracyScore_2 * 100).round(0)) + '%')
print('ROC AUC Score: ' + str((RocAucScore_2 * 100).round(0)) + '%')


In [ ]:
######## Additional Reports ########
### Confusion Matrix
## A confusion matrix is a summary of prediction results on a classification problem
## The number of correct and incorrect predictions are summarised with count values

## true prediction | false prediction
## false prediction | true prediction

print('Confusion Matrix')
print(metrics.confusion_matrix(y_test, predicted))
print()
### Classification Report
print('Classification Report')
print(metrics.classification_report(y_test, predicted))

In [ ]:
######################################################################
########                  Model Validation                    ########
######################################################################

### Evaluate the model using 10-fold cross-validation
scores = cross_val_score(LogisticRegression(), x, y, scoring = 'accuracy', cv = 10)
print('Cross Validation Scores')
print((scores * 100).round(0))
print()
print('Average Scores')
print((scores.mean() * 100).round(0))

### Model is still performing at 72% accuracy

In [ ]:
######################################################################
########                    Next Steps                        ########
######################################################################

### There are many different steps that could be tried in order to improve the model:
# including interaction terms
# removing features
# regularization techniques
# using a non-linear model